In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import string
import nltk
import pickle

In [ ]:
import os
import pandas as pd
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
os.chdir("/content/gdrive/My Drive/Practice module NLP/PLP Practice Module/hwz")

Mounted at /content/gdrive


# Extract all thread titles, urls in ALL pages.

## Extract Page 1 thread titles and urls only

In [ ]:
URL = "https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(class_="structItemContainer-group js-threadList")


In [ ]:
job_elements = results.find_all("div", class_="structItem-title")
for job_element in job_elements:
    print(job_element, end="\n"*2)

<div class="structItem-title">
<a class="" data-preview-url="/threads/lai-liao-europe-covid-surges-again.6639742/preview" data-tp-primary="on" data-xf-init="preview-tooltip" href="/threads/lai-liao-europe-covid-surges-again.6639742/">Lai liao!! Europe covid surges again!!!</a>
</div>

<div class="structItem-title">
<a class="" data-preview-url="/threads/singapores-unvaccinated-may-face-covid-medical-bills-over-s-25-000.6639743/preview" data-tp-primary="on" data-xf-init="preview-tooltip" href="/threads/singapores-unvaccinated-may-face-covid-medical-bills-over-s-25-000.6639743/">Singapore’s unvaccinated may face Covid medical bills over S$25,000!</a>
</div>

<div class="structItem-title">
<a class="" data-preview-url="/threads/just-had-my-second-pfizer-vaccination-with-my-mum.6625981/preview" data-tp-primary="on" data-xf-init="preview-tooltip" href="/threads/just-had-my-second-pfizer-vaccination-with-my-mum.6625981/">Just had my second Pfizer vaccination with my mum.</a>
</div>

<div cla

In [ ]:
for job_element in job_elements:
    links = job_element.find_all("a")
    for link in links:
        link_url = link["href"]
        link_url = "https://forums.hardwarezone.com.sg"+link_url
        title = link.text.strip()
        print(f"thread_url: {link_url}")
        print(f"thread: {title}\n")

thread_url: https://forums.hardwarezone.com.sg/threads/lai-liao-europe-covid-surges-again.6639742/
thread: Lai liao!! Europe covid surges again!!!

thread_url: https://forums.hardwarezone.com.sg/threads/singapores-unvaccinated-may-face-covid-medical-bills-over-s-25-000.6639743/
thread: Singapore’s unvaccinated may face Covid medical bills over S$25,000!

thread_url: https://forums.hardwarezone.com.sg/threads/just-had-my-second-pfizer-vaccination-with-my-mum.6625981/
thread: Just had my second Pfizer vaccination with my mum.

thread_url: https://forums.hardwarezone.com.sg/threads/its-now-about-how-big-or-how-nice-your-home-is-but-who-lives-inside-it-with-you-its-not-about-how-nice-your-car-is-but-who-rides-in-the-car-with-u.6639704/
thread: its now about how big or how nice your home is, but who lives inside it with you. its not about how nice your car is, but who rides in the car with u

thread_url: https://forums.hardwarezone.com.sg/threads/song-boh-govt-to-review-when-fully-vaccinate

##Function to extract thread titles and urls in each page

In [ ]:
def extractthread(url):
  #forum_cols = ['thread', 'thread_url']
  forum_df = []
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  results = soup.find(class_="structItemContainer-group js-threadList")
  job_elements = results.find_all("div", class_="structItem-title")
  for job_element in job_elements:
    links = job_element.find_all("a")
    for link in links:
        link_url = link["href"]
        link_url = "https://forums.hardwarezone.com.sg"+link_url
        title = link.text.strip()
    row = (title, link_url)
    forum_df.append(row)
  return forum_df

In [ ]:
extractthread("https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/")

[('KNN! My wife suddenly text me say if i  not vaccinated by 2359 tonight, cannot share the same bed and dining table',
  'https://forums.hardwarezone.com.sg/threads/knn-my-wife-suddenly-text-me-say-if-i-not-vaccinated-by-2359-tonight-cannot-share-the-same-bed-and-dining-table.6621916/'),
 ('Riots? Deport the foreign workers.',
  'https://forums.hardwarezone.com.sg/threads/riots-deport-the-foreign-workers.6621914/'),
 ('Those people that join Grassroot is really wanna help and serve e people or got other motive like',
  'https://forums.hardwarezone.com.sg/threads/those-people-that-join-grassroot-is-really-wanna-help-and-serve-e-people-or-got-other-motive-like.6621926/'),
 ('2932 CASES, 15 DEATH CASES 23 Y.O & 34 Y.O DIES',
  'https://forums.hardwarezone.com.sg/threads/2932-cases-15-death-cases-23-y-o-34-y-o-dies.6621884/'),
 ('If marry liao still need to wear protection how?',
  'https://forums.hardwarezone.com.sg/threads/if-marry-liao-still-need-to-wear-protection-how.6621871/'),
 ('G

In [ ]:
len(extractthread("https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/"))

20

## Extract ALL pages'  thread titles and URLs

In [ ]:
from time import sleep
from tqdm import tqdm
threadsdf =[]
for pagenumber in tqdm(range(3000,4000)):
  URL = "https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/page-"+ str(pagenumber)
  page = extractthread(URL)
  threadsdf.append(page)


100%|██████████| 1000/1000 [20:29<00:00,  1.23s/it]


In [ ]:
listofthreads = [e for l in threadsdf for e in l]


In [ ]:
len(listofthreads)

20000

In [ ]:
df = pd.DataFrame(listofthreads, columns =['header', 'url'])
df.head(4)

,header,url
0,just now went macronalds saw one cute trainee,https://forums.hardwarezone.com.sg/threads/jus...
1,Singapore should ban such axe throwing busines...,https://forums.hardwarezone.com.sg/threads/sin...
2,[Breaking] Man who scraped LinkedIn's 700M pro...,https://forums.hardwarezone.com.sg/threads/bre...
3,"Ate mcspicy for supper yesterday, what time wi...",https://forums.hardwarezone.com.sg/threads/ate...


# Extract only covid threads

## Import files

In [ ]:
df1 = pd.read_csv('1000.csv',index_col=0)
df2 = pd.read_csv('2000.csv',index_col=0)
df3 = pd.read_csv('3000.csv',index_col=0)
df4 = pd.read_csv('4000.csv',index_col=0)
df5 = pd.read_csv('5000.csv',index_col=0)
df6 = pd.read_csv('6000.csv',index_col=0)
df7 = pd.read_csv('7000.csv',index_col=0)
df8 = pd.read_csv('8000.csv',index_col=0)
df9 = pd.read_csv('9000.csv',index_col=0)
df10 = pd.read_csv('10000.csv',index_col=0)

df = pd.concat([df1,df2, df3, df4, df5, df6, df7, df8, df9, df10])
print(f"Number of threads: {len(df)}")

Number of threads: 184716


In [ ]:
df = df.reset_index(drop=True)
df= df.drop_duplicates(keep='first').reset_index(drop=True)

In [ ]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
WNlemma = nltk.WordNetLemmatizer()
def pre_process(text):
    text = str(text)
    #text = " ".join(re.sub("[.,!?:;-='...'@]"," ",text).split())
    text = deEmojify(text)
    tokens = nltk.word_tokenize(text)
    tokens = [ t for t in tokens if t not in string.punctuation ]
    tokens = [WNlemma.lemmatize(t.lower()) for t in tokens]
    #tokens = nltk.word_tokenize(text)
    text_after_process=" ".join(tokens)
    return(text_after_process)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df["clean_header"] = df["header"].apply(pre_process)
df["clean_header"][371]  #unable to remove - in post-covid-19

"s'pore china can team up to access new opportunity in three area post-covid-19 low yen ling"

In [ ]:
print(df[['header','clean_header']])

                                                   header                                       clean_header
0       Tan See Leng called unhappy chinese migrant wo...  tan see leng called unhappy chinese migrant wo...
1            Wanbao will be closing shop year end... kym?       wanbao will be closing shop year end ... kym
2                              Who is AYAYAngry? - Part 4                            who is ayayangry part 4
3                                  Chit Chat with Mommy 😊                               chit chat with mommy
4       Chinese Embassy: We will continue to track dev...  chinese embassy we will continue to track deve...
...                                                   ...                                                ...
169955                  Why Medic also kenna charged? >:(                       why medic also kenna charged
169956  Fire engulfs Sembawang God of Wealth leaves Go...  fire engulfs sembawang god of wealth leaf god ...
169957         fu**

## Detect covid words

In [ ]:
covidwords = ['covid', 'covid19','covid 19','covid-19','coronavirus','corona','china virus',
              'precovid', 'precovid19','precovid 19','precovid-19',
              'pre-covid', 'pre-covid19','pre-covid 19','pre-covid-19',
              'postcovid', 'postcovid19','postcovid 19','postcovid-19',
              'post-covid', 'post-covid19','post-covid 19','post-covid-19']

def detect_covid(text):
    text = re.sub(r"[,.;@#?!-&$]+\ *", " ", str(text))
    tokens = nltk.word_tokenize(text)
    tokens = [ t for t in tokens if t  in covidwords ]
    text_after_process=" ".join(tokens)
    return(text_after_process)

In [ ]:
df["covidheaders"] = df["clean_header"].apply(detect_covid)

In [ ]:
print(df['header'][371])
print(df['clean_header'][371])
print(df['covidheaders'][371])

S'pore, China can team up to access new opportunities in three areas post-Covid-19: Low Yen Ling
s'pore china can team up to access new opportunity in three area post-covid-19 low yen ling
post-covid-19


In [ ]:
#total number of thread titles with covid headers
df[df['covidheaders']!=''].count()

header          11277
url             11277
clean_header    11277
covidheaders    11277
dtype: int64

In [ ]:
df_covid = df[df['covidheaders']!='']
df_covid.reset_index(inplace=True, drop =True)

In [ ]:
df_covid.head(5)

,header,url,clean_header,covidheaders
0,"Dr Ooi, Duke-NUS Prof: We have to stop fearing...",https://forums.hardwarezone.com.sg/threads/dr-...,dr ooi duke-nus prof we have to stop fearing c...,covid
1,Congrat me! Tio Covid TOTO,https://forums.hardwarezone.com.sg/threads/con...,congrat me tio covid toto,covid
2,Security guard who flouted stay-home order whi...,https://forums.hardwarezone.com.sg/threads/sec...,security guard who flouted stay-home order whi...,covid-19
3,How messy is MOH arrangement upon the surge of...,https://forums.hardwarezone.com.sg/threads/how...,how messy is moh arrangement upon the surge of...,covid
4,MOM investigates claims about COVID-19 health ...,https://forums.hardwarezone.com.sg/threads/mom...,mom investigates claim about covid-19 health b...,covid-19


In [ ]:
#df_covid.to_csv('df_covid.csv')

# Function to extract contents from each page of thread url

In [ ]:
def getPageSoup(URL):
  '''
		Opens connection and gets page soup
		Args: url
	'''
  
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  return soup

In [ ]:
def getPageNumbers(page_soup):
	'''
		Gets page numbers from soup object.
		Args: page_soup
	'''
	y = page_soup.find("a", {"class": "pageNavSimple-el pageNavSimple-el--current"})
	if y is None:
		# this means that it is single page !
		total_pages = 1
	else:
		total_pages = int(y.text.split(" ")[-1])
	return total_pages

In [ ]:
def getPosts(page_soup):
	'''
		Gets all posts from soup object.
		Args: page_soup
	'''
	return page_soup.find_all("div", {"class": "bbWrapper"})

In [ ]:
def getDatetime(page_soup):
  return page_soup.findAll('time')


In [ ]:
def getUsername(page_soup):
  return page_soup.find_all("h4", {"class": "message-name"})

# Create url_contents list with dictionary: username, datetime, content

In [ ]:
total_count = 0

# change the range to extract the different links in df_covid
for x in range(0,10):
  url = df_covid['url'][x]
  total_pages = getPageNumbers(getPageSoup(url))
  pg = 1
  count = 0
  username_list = []
  content_list = []
  datetime_list = []

  #Skip these pages due to errors and duplicates of previous posts
  if total_pages == 124 or total_pages == 125: 
    content_list.append("none")
    username_list.append("none")
    datetime_list.append("none")
    df = {'datetime': datetime_list,
          'username': username_list,
          'content': content_list}

    url_contents[x]=df
    continue

  else:
      while pg <= total_pages:
            print("\nScrapping page: ", pg, "....\n")
            url_loop = url + 'page-' + str(pg)
            pg += 1

            page_soup = getPageSoup(url_loop)
            posts = getPosts(page_soup)
            datetimes = getDatetime(page_soup)
            usernames = getUsername(page_soup)
            
            for post,username,datetime in zip(posts,usernames,datetimes):
              count += 1
              total_count += 1
              post_content = post.text.strip()
              username_content = username.text.strip()

              content_list.append(post_content)
              username_list.append(username_content)

              if datetime.has_attr('datetime'):
                datetime_list.append(datetime['datetime'])
          
              df = {'datetime': datetime_list,
                    'username': username_list,
                    'content': content_list}
              print("Scrapped link %s,  %s post. Post: %d, Cumulative post: %d" % (x,username.text.strip(), count, total_count))
      url_contents[x]=df

print(f"Total cumulative comments: {total_count}")

In [ ]:
df_covid = pd.read_csv('df_covid.csv',index_col=0)
print(f"Number of urls: {len(df_covid)}")

Number of urls: 11277


In [ ]:
with open('url_contents11277.pickle', 'rb') as handle:
  url_contents = pickle.load(handle)

In [ ]:
with open('url_contents11277.pickle', 'wb') as handle:
    pickle.dump(url_contents, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Total Comment count: 403139

In [ ]:
comments_count = 0
for i in range(0, len(url_contents)):
  print(f"Number of comments in URL {i}: {len(url_contents[i]['content'])}")
  comments_count += len(url_contents[i]['content'])
print(f"Total number of comments: {comments_count}")

Streaming output truncated to the last 5000 lines.
Number of comments in URL 6278: 1
Number of comments in URL 6279: 2
Number of comments in URL 6280: 7
Number of comments in URL 6281: 6
Number of comments in URL 6282: 26
Number of comments in URL 6283: 5
Number of comments in URL 6284: 1
Number of comments in URL 6285: 2
Number of comments in URL 6286: 1
Number of comments in URL 6287: 87
Number of comments in URL 6288: 2
Number of comments in URL 6289: 4
Number of comments in URL 6290: 48
Number of comments in URL 6291: 19
Number of comments in URL 6292: 1
Number of comments in URL 6293: 33
Number of comments in URL 6294: 4
Number of comments in URL 6295: 5
Number of comments in URL 6296: 6
Number of comments in URL 6297: 6
Number of comments in URL 6298: 53
Number of comments in URL 6299: 1
Number of comments in URL 6300: 4
Number of comments in URL 6301: 57
Number of comments in URL 6302: 1
Number of comments in URL 6303: 1
Number of comments in URL 6304: 2
Number of comments in UR

## Total Date count: 403139

In [ ]:
date_count = 0
for i in range(0, len(url_contents)):
  print(f"Number of dates in URL {i}: {len(url_contents[i]['datetime'])}")
  date_count += len(url_contents[i]['datetime'])
print(f"Total number of comments: {date_count}")

Streaming output truncated to the last 5000 lines.
Number of dates in URL 6278: 1
Number of dates in URL 6279: 2
Number of dates in URL 6280: 7
Number of dates in URL 6281: 6
Number of dates in URL 6282: 26
Number of dates in URL 6283: 5
Number of dates in URL 6284: 1
Number of dates in URL 6285: 2
Number of dates in URL 6286: 1
Number of dates in URL 6287: 87
Number of dates in URL 6288: 2
Number of dates in URL 6289: 4
Number of dates in URL 6290: 48
Number of dates in URL 6291: 19
Number of dates in URL 6292: 1
Number of dates in URL 6293: 33
Number of dates in URL 6294: 4
Number of dates in URL 6295: 5
Number of dates in URL 6296: 6
Number of dates in URL 6297: 6
Number of dates in URL 6298: 53
Number of dates in URL 6299: 1
Number of dates in URL 6300: 4
Number of dates in URL 6301: 57
Number of dates in URL 6302: 1
Number of dates in URL 6303: 1
Number of dates in URL 6304: 2
Number of dates in URL 6305: 30
Number of dates in URL 6306: 3
Number of dates in URL 6307: 3
Number of d

## Total Username count: 403139

In [ ]:
username_count = 0
for i in range(0, len(url_contents)):
  print(f"Number of username in URL {i}: {len(url_contents[i]['username'])}")
  username_count += len(url_contents[i]['username'])
print(f"Total number of username: {username_count}")

Streaming output truncated to the last 5000 lines.
Number of username in URL 6278: 1
Number of username in URL 6279: 2
Number of username in URL 6280: 7
Number of username in URL 6281: 6
Number of username in URL 6282: 26
Number of username in URL 6283: 5
Number of username in URL 6284: 1
Number of username in URL 6285: 2
Number of username in URL 6286: 1
Number of username in URL 6287: 87
Number of username in URL 6288: 2
Number of username in URL 6289: 4
Number of username in URL 6290: 48
Number of username in URL 6291: 19
Number of username in URL 6292: 1
Number of username in URL 6293: 33
Number of username in URL 6294: 4
Number of username in URL 6295: 5
Number of username in URL 6296: 6
Number of username in URL 6297: 6
Number of username in URL 6298: 53
Number of username in URL 6299: 1
Number of username in URL 6300: 4
Number of username in URL 6301: 57
Number of username in URL 6302: 1
Number of username in URL 6303: 1
Number of username in URL 6304: 2
Number of username in UR

#Creating contents df only from url_contents

In [ ]:
content = []
url_link = []
url_header = []
for url_no in range(0, len(url_contents)):
  for comment_no in range(0,len(url_contents[url_no]['content'])):
    #print(f"Comment in URL {url_no}, current count {comment_no+1} out of {len(url_contents[url_no]['content'])}:\n {url_contents[url_no]['content'][comment_no]}\n")
    print(f"Comment in URL {url_no}, current count {comment_no+1} out of {len(url_contents[url_no]['content'])}")
    content.append(url_contents[url_no]['content'][comment_no])
    url_link.append(df_covid['header'][url_no])
    url_header.append(df_covid['url'][url_no])

Streaming output truncated to the last 5000 lines.
Comment in URL 11241, current count 542 out of 577
Comment in URL 11241, current count 543 out of 577
Comment in URL 11241, current count 544 out of 577
Comment in URL 11241, current count 545 out of 577
Comment in URL 11241, current count 546 out of 577
Comment in URL 11241, current count 547 out of 577
Comment in URL 11241, current count 548 out of 577
Comment in URL 11241, current count 549 out of 577
Comment in URL 11241, current count 550 out of 577
Comment in URL 11241, current count 551 out of 577
Comment in URL 11241, current count 552 out of 577
Comment in URL 11241, current count 553 out of 577
Comment in URL 11241, current count 554 out of 577
Comment in URL 11241, current count 555 out of 577
Comment in URL 11241, current count 556 out of 577
Comment in URL 11241, current count 557 out of 577
Comment in URL 11241, current count 558 out of 577
Comment in URL 11241, current count 559 out of 577
Comment in URL 11241, current c

In [ ]:
len(content)

403139

#Creating datetime df only from url_contents

In [ ]:
datetime = []
for url_no in range(0, len(url_contents)):
  for datetime_no in range(0,len(url_contents[url_no]['datetime'])):
    print(f"Datetime in URL {url_no}, current count {datetime_no+1} out of {len(url_contents[url_no]['datetime'])}")
    datetime.append(url_contents[url_no]['datetime'][datetime_no])

Streaming output truncated to the last 5000 lines.
Datetime in URL 11241, current count 542 out of 577
Datetime in URL 11241, current count 543 out of 577
Datetime in URL 11241, current count 544 out of 577
Datetime in URL 11241, current count 545 out of 577
Datetime in URL 11241, current count 546 out of 577
Datetime in URL 11241, current count 547 out of 577
Datetime in URL 11241, current count 548 out of 577
Datetime in URL 11241, current count 549 out of 577
Datetime in URL 11241, current count 550 out of 577
Datetime in URL 11241, current count 551 out of 577
Datetime in URL 11241, current count 552 out of 577
Datetime in URL 11241, current count 553 out of 577
Datetime in URL 11241, current count 554 out of 577
Datetime in URL 11241, current count 555 out of 577
Datetime in URL 11241, current count 556 out of 577
Datetime in URL 11241, current count 557 out of 577
Datetime in URL 11241, current count 558 out of 577
Datetime in URL 11241, current count 559 out of 577
Datetime in U

In [ ]:
len(datetime)

403139

#Creating username df only from url_contents

In [ ]:
username = []
for url_no in range(0, len(url_contents)):
  for username_no in range(0,len(url_contents[url_no]['username'])):
    print(f"Username in URL {url_no}, current count {username_no+1} out of {len(url_contents[url_no]['username'])}")
    username.append(url_contents[url_no]['username'][username_no])

Streaming output truncated to the last 5000 lines.
Username in URL 11241, current count 542 out of 577
Username in URL 11241, current count 543 out of 577
Username in URL 11241, current count 544 out of 577
Username in URL 11241, current count 545 out of 577
Username in URL 11241, current count 546 out of 577
Username in URL 11241, current count 547 out of 577
Username in URL 11241, current count 548 out of 577
Username in URL 11241, current count 549 out of 577
Username in URL 11241, current count 550 out of 577
Username in URL 11241, current count 551 out of 577
Username in URL 11241, current count 552 out of 577
Username in URL 11241, current count 553 out of 577
Username in URL 11241, current count 554 out of 577
Username in URL 11241, current count 555 out of 577
Username in URL 11241, current count 556 out of 577
Username in URL 11241, current count 557 out of 577
Username in URL 11241, current count 558 out of 577
Username in URL 11241, current count 559 out of 577
Username in U

In [ ]:
len(username)

403139

#Final dataset

In [ ]:
df_covid

,header,url,clean_header,covidheaders
0,"Dr Ooi, Duke-NUS Prof: We have to stop fearing...",https://forums.hardwarezone.com.sg/threads/dr-...,dr ooi duke-nus prof we have to stop fearing c...,covid
1,Congrat me! Tio Covid TOTO,https://forums.hardwarezone.com.sg/threads/con...,congrat me tio covid toto,covid
2,Security guard who flouted stay-home order whi...,https://forums.hardwarezone.com.sg/threads/sec...,security guard who flouted stay-home order whi...,covid-19
3,How messy is MOH arrangement upon the surge of...,https://forums.hardwarezone.com.sg/threads/how...,how messy is moh arrangement upon the surge of...,covid
4,MOM investigates claims about COVID-19 health ...,https://forums.hardwarezone.com.sg/threads/mom...,mom investigates claim about covid-19 health b...,covid-19
...,...,...,...,...
11272,"Man fined S$3,000 for gathering with 11 others...",https://forums.hardwarezone.com.sg/threads/man...,"man fined s 3,000 for gathering with 11 others...",covid-19
11273,[Latest]Covid-19 News: Singapore reports 28th ...,https://forums.hardwarezone.com.sg/threads/lat...,latest covid-19 news singapore report 28th death,covid-19
11274,Twelve people charged over unlawful gathering ...,https://forums.hardwarezone.com.sg/threads/twe...,twelve people charged over unlawful gathering ...,covid-19
11275,"trump tio covid, what's your first thought on ...",https://forums.hardwarezone.com.sg/threads/tru...,trump tio covid what 's your first thought on ...,covid


In [ ]:
data = pd.DataFrame(list(zip(url_header, url_link, datetime, username, content)), columns = ['header','url','datetime','username','content'])

In [ ]:
data_remove = data.drop_duplicates(keep='first')
len(data_remove)

305206

In [ ]:
print(f"Total number of duplicates removed: {len(data)-len(data_remove)}")
print(f"Total number of comments: {len(data_remove)}")

Total number of duplicates removed: 97933
Total number of comments: 305206


In [ ]:
data_remove

,header,url,datetime,username,content
0,https://forums.hardwarezone.com.sg/threads/dr-...,"Dr Ooi, Duke-NUS Prof: We have to stop fearing...",2021-10-14T21:35:38+0800,tripleme,
1,https://forums.hardwarezone.com.sg/threads/dr-...,"Dr Ooi, Duke-NUS Prof: We have to stop fearing...",2021-10-14T21:35:38+0800,guile,These experts should tell this straight to fac...
2,https://forums.hardwarezone.com.sg/threads/dr-...,"Dr Ooi, Duke-NUS Prof: We have to stop fearing...",2021-10-14T21:37:04+0800,trento,tripleme said:\n\n\n\n\n\nClick to expand...\n...
3,https://forums.hardwarezone.com.sg/threads/dr-...,"Dr Ooi, Duke-NUS Prof: We have to stop fearing...",2021-10-14T21:39:54+0800,powerrr,Stop fearing Covid? Is PAP’s MPS cancelled? If...
4,https://forums.hardwarezone.com.sg/threads/dr-...,"Dr Ooi, Duke-NUS Prof: We have to stop fearing...",2021-10-14T21:40:29+0800,IAMTOXIC,TALK IS CHEAP...\nHOW IS HE CONTRIBUTING?\nhis...
...,...,...,...,...,...
403134,https://forums.hardwarezone.com.sg/threads/32-...,32 diners to be fined $300 each for flouting C...,2021-10-17T13:58:22+0800,limpeh456,SOS9112001 said:\n\n\n\n\t\t\tWow. I didn't kn...
403135,https://forums.hardwarezone.com.sg/threads/32-...,32 diners to be fined $300 each for flouting C...,2021-10-17T14:03:37+0800,roary9,iamnew said:\n\n\n\n\t\t\tSometimes I really w...
403136,https://forums.hardwarezone.com.sg/threads/32-...,32 diners to be fined $300 each for flouting C...,2021-10-17T14:08:42+0800,TurboBumbleBee,Dine in until so kang kor…… they want u to din...
403137,https://forums.hardwarezone.com.sg/threads/32-...,32 diners to be fined $300 each for flouting C...,2020-09-30T08:16:03+0800,cloudnim,$300 each can fine many many everyday one


In [ ]:
data_remove.to_csv('data_remove.csv')

Scrape Hardwarezone references



https://github.com/cheryl-ngxy/project-forum/

https://github.com/jaabberwocky/hwzscrapper